In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competicao-um-ic/train.csv
/kaggle/input/competicao-um-ic/test.csv


In [2]:
df_train = pd.read_csv('/kaggle/input/competicao-um-ic/train.csv')
df_test = pd.read_csv('/kaggle/input/competicao-um-ic/test.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def preproc(df):
    df.QT_DIA_SOLICITADO = df.QT_DIA_SOLICITADO.fillna(0)
    df.DS_INDICACAO_ACIDENTE = df.DS_INDICACAO_ACIDENTE.fillna('0')
    df.DS_TIPO_INTERNACAO = df.DS_TIPO_INTERNACAO.fillna('0')
    df.DS_TIPO_ATENDIMENTO = df.DS_TIPO_ATENDIMENTO.fillna('0')
    df.DS_UNIDADE_TEMPO_DOENCA = df.DS_UNIDADE_TEMPO_DOENCA.fillna('0')
    df.QT_TEMPO_DOENCA = df.QT_TEMPO_DOENCA.fillna(0)
    df.CD_GUIA_REFERENCIA = df.CD_GUIA_REFERENCIA.fillna(0)
    return df

In [4]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import numpy as np

def standardScalerFunc(data):
  ss = StandardScaler()
  sstransformed = ss.fit_transform(data)

  return ss, pd.DataFrame(sstransformed)

def labelEncoderFunc(data):
  ss = LabelEncoder()
  sstransformed = ss.fit_transform(data) 

  return ss, pd.DataFrame(sstransformed)

def oneHotEncoderFunc(data):
  ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
  ohetransformed = ohe.fit_transform(data)
  
  return ohe, pd.DataFrame(ohetransformed)

In [5]:
df_train = preproc(df_train)
numeric = df_train[['NR_SEQ_REQUISICAO', 'QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']]
categoric = df_train[['DS_ITEM','DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE', 'DS_TIPO_GUIA', 'DS_GRUPO', 'CD_ITEM', 'DS_CBO', 'DS_SUBGRUPO']]
target = df_train['DS_STATUS_ITEM']

X = pd.concat([numeric, categoric], axis=1)
print(len(X))
X = X.dropna()
print(len(X))
y = target

227122
227122


In [6]:
# df_test['DS_STATUS_ITEM'] = ['Negado']*len(df_test)
# df_test['ID'] = df_test['Unnamed: 0']
# df_test[['ID','DS_STATUS_ITEM']].set_index('ID').to_csv('submission.csv')

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 80% training 20% test

#train
ss, x_train_ss = standardScalerFunc(X_train[['QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']])
ohe, x_train_ohe = oneHotEncoderFunc(X_train[['DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE', 'DS_TIPO_GUIA', 'DS_GRUPO']])

x_train = pd.concat([x_train_ss, x_train_ohe], axis=1)

#test
x_test_ss = ss.transform(X_test[['QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']])
x_test_ohe = ohe.transform(X_test[['DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE', 'DS_TIPO_GUIA', 'DS_GRUPO']])

x_test = pd.concat([pd.DataFrame(x_test_ss), pd.DataFrame(x_test_ohe)], axis=1)

#classifier
classifier = RandomForestClassifier(max_depth=20, max_features=10, min_samples_leaf=3)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

In [8]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[29333  1499]
 [11224  3369]]
              precision    recall  f1-score   support

  Autorizado       0.72      0.95      0.82     30832
      Negado       0.69      0.23      0.35     14593

    accuracy                           0.72     45425
   macro avg       0.71      0.59      0.58     45425
weighted avg       0.71      0.72      0.67     45425

0.7199119427627958


In [9]:
df_test_pp = preproc(df_test)
test_ss = ss.transform(df_test_pp[['QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']])
test_ohe = ohe.transform(df_test_pp[['DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE', 'DS_TIPO_GUIA', 'DS_GRUPO']])

test = pd.concat([pd.DataFrame(test_ss), pd.DataFrame(test_ohe)], axis=1)

In [10]:
pred = classifier.predict(test)

In [11]:
df_test

,Unnamed: 0,NR_SEQ_REQUISICAO,NR_SEQ_ITEM,DT_REQUISICAO,DS_TIPO_GUIA,DT_NASCIMENTO,NR_PRODUTO,DS_TIPO_PREST_SOLICITANTE,DS_CBO,DS_TIPO_CONSULTA,...,DS_TIPO_ATENDIMENTO,CD_CID,DS_INDICACAO_CLINICA,DS_TIPO_ITEM,CD_ITEM,DS_ITEM,DS_CLASSE,DS_SUBGRUPO,DS_GRUPO,QT_SOLICITADA
0,0,1560479,4348670,2459598,Guia de solicitac?o SP/SADT,2440511.0,1,HOSPITAL,Medico ortopedista e traumatologista,NaN,...,Pronto socorro,NaN,NaN,Procedimentos,60023104,PORTE 1 (CIRURGIA PEQUENA),EM CENTRO CIRURGICO,TAXA DE SALA,TAXAS,1.0
1,5,1351610,937069,2459487,Guia de solicitac?o SP/SADT,2441089.0,1,HOSPITAL,Medico clinico,NaN,...,Quimioterapia,C50,C50,MAT/MED,73512,ZEDORA 440 MG PO LIOF SOL INJ IV CT FA VD TRAN...,ANTINEOPLASICOS,AGENTES ANTINEOPLASICOS,Brasindice,880.0
2,8,1559916,4347132,2459598,Guia de solicitac?o SP/SADT,2439094.0,1,CLINICA DE IMAGEM,Medico clinico,NaN,...,Exames,NaN,NaN,Procedimentos,41001079,TC - TOMOGRAFIA COMPUTADORIZADA -TORAX,TOMOGRAFIA COMPUTADORIZADA DIAGNOSTICA,TOMOGRAFIA COMPUTADORIZADA,PROCED. DIAGNOSTICOS E TERAPEUTICOS,1.0
3,13,1516924,960836,2459571,Guia de solicitac?o SP/SADT,2442178.0,1,CLINICA,Medico ortopedista e traumatologista,NaN,...,Pequenos atendimentos,NaN,NaN,MAT/MED,162121,ATADURA GESSADA CYSNE 10CMX3MT - CM,Material de consumo,Material Hospitalar,Simpro,600.0
4,18,1484086,4111308,2459555,Guia de solicitac?o SP/SADT,2436205.0,1,CLINICA DE IMAGEM,Medico endocrinologista e metabologista,NaN,...,Exames,NaN,EM ANEXO,Procedimentos,40901203,USG - ULTRASSONOGRAFIA - ORG?OS SUPERFICIAIS (...,ULTRA-SONOGRAFIA DIAGNOSTICA,ULTRA-SONOGRAFIA,PROCED. DIAGNOSTICOS E TERAPEUTICOS,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186139,413210,1308999,3543118,2459467,Guia de solicitac?o SP/SADT,2434126.0,1,MEDICO,Medico oftalmologista,Primeira consulta,...,Exames,H40,EM ANEXO,Procedimentos,41401301,TESTE PROVOCATIVO PARA GLAUCOMA - BINOCULAR,PROCEDIMENTOS,TESTES PARA DIAGNOSTICO,PROCED. DIAGNOSTICOS E TERAPEUTICOS,1.0
186140,413218,1309100,928486,2459467,Guia de solicitac?o internac?o,2436152.0,1,HOSPITAL,Medico urologista,NaN,...,0,NaN,Paciente com calculo ureteral proximal a esque...,MAT/MED,276659,"FIO GUIA 0,014""X300CM HIDROFILICO STD J/CP STI...",Protese,Material Hospitalar,Simpro,2.0
186141,413221,1318518,3574140,2459472,Guia de solicitac?o SP/SADT,2451525.0,1,CLINICA,Medico ortopedista e traumatologista,NaN,...,Pronto socorro,NaN,PCT VITIMA DE QUEDA DE MOTO COM ESCORIAC?ES+ F...,Procedimentos,30711010,IMOBILIZAC?ES N?O-GESSADAS (QUALQUER SEGMENTO),IMOBILIZAC?ES PROVISORIAS - TALAS GESSADAS,SISTEMA MUSCULO-ESQUELETICO E ARTICULAC?ES,PROCED. CIRURGICOS E INVASIVOS,1.0
186142,413248,1300775,3515098,2459460,Guia de solicitac?o internac?o,2434417.0,1,HOSPITAL,Medico ortopedista e traumatologista,NaN,...,0,NaN,SINDROME DO TUNEL DO CARPO BILATERAL,Procedimentos,60000783,ENFERMARIA,DIARIAS,DIARIAS,DIARIAS,1.0


In [12]:
from itertools import zip_longest
submission = pd.DataFrame.from_records(zip_longest(df_test['Unnamed: 0'], pred), columns=['ID', 'DS_STATUS_ITEM'])

submission.set_index('ID').to_csv('submission.csv')

In [13]:
submission

,ID,DS_STATUS_ITEM
0,0,Negado
1,5,Autorizado
2,8,Autorizado
3,13,Autorizado
4,18,Autorizado
...,...,...
186139,413210,Autorizado
186140,413218,Autorizado
186141,413221,Negado
186142,413248,Autorizado
